## Lab | Random Forests
For this lab, you will be using the CSV files provided in the files_for_lab folder.

Instructions
- Apply the Random Forests algorithm but this time only by upscaling the data using SMOTE.
- Note that since SMOTE works on numerical data only, we will first encode the categorical variables in this case.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

In [2]:
num=pd.read_csv('numerical (3).csv')
cat=pd.read_csv('categorical.csv')
target=pd.read_csv('target.csv')

In [3]:
data=pd.concat([num, cat], axis=1)
data

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
0,0,60.000000,5,9,0,0,39,34,18,10,...,37,12,92,8,94,2,95,12,89,11
1,1,46.000000,6,9,16,0,15,55,11,6,...,52,2,93,10,95,12,95,12,93,10
2,1,61.611649,3,1,2,0,20,29,33,6,...,0,2,91,11,92,7,95,12,90,1
3,0,70.000000,1,4,2,0,23,14,31,3,...,28,1,87,11,94,11,95,12,87,2
4,0,78.000000,3,2,60,1,28,9,53,26,...,20,1,93,10,96,1,96,1,79,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,1,61.611649,5,9,0,14,36,47,11,7,...,0,2,96,2,96,2,96,2,96,2
95408,1,48.000000,7,9,1,0,31,43,19,4,...,50,1,96,3,96,3,96,3,96,3
95409,1,60.000000,5,9,0,0,18,46,20,7,...,38,1,96,3,95,1,96,10,94,10
95410,0,58.000000,7,9,0,0,28,35,20,9,...,40,5,90,11,96,8,97,1,86,12


In [4]:
 #Vamos a hacer una matriz de correlacion para eliminar las columnas con una correlacion alta
corr_matrix=data.corr()

In [5]:
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(bool))

In [6]:
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.80)] # Eliminamos las columnas con una correlacion mas alta al 80%\n",
print(to_drop)

['POP902', 'POP903', 'ETH13', 'AGE902', 'AGE903', 'AGE904', 'AGE905', 'AGE906', 'AGEC6', 'CHILC1', 'CHILC3', 'HHAGE1', 'HHAGE2', 'HHAGE3', 'HHN3', 'HHN4', 'HHN5', 'HHN6', 'MARR3', 'HHP1', 'HHP2', 'DW2', 'DW5', 'DW6', 'HV2', 'HV4', 'HU2', 'HHD1', 'HHD2', 'HHD3', 'HHD4', 'HHD5', 'HHD6', 'HHD9', 'HHD11', 'ETHC2', 'ETHC3', 'ETHC4', 'ETHC5', 'ETHC6', 'HVP1', 'HVP2', 'HVP3', 'HVP4', 'HVP5', 'HVP6', 'RHP1', 'RHP2', 'RHP3', 'HUPA2', 'HUPA5', 'HUPA6', 'RP1', 'RP2', 'RP3', 'RP4', 'IC2', 'IC3', 'IC4', 'IC5', 'IC11', 'IC14', 'IC15', 'IC16', 'IC17', 'IC18', 'IC19', 'IC20', 'IC21', 'IC22', 'IC23', 'HHAS1', 'HHAS4', 'TPE4', 'LFC2', 'LFC3', 'LFC4', 'LFC5', 'EIC1', 'OEDC1', 'OEDC2', 'OEDC3', 'SEC4', 'AFC2', 'AFC3', 'AFC5', 'LSC2', 'LSC3', 'VOC2', 'VOC3', 'HC5', 'HC6', 'HC7', 'HC8', 'HC21', 'NUMPROM', 'CARDGIFT', 'FIRSTDATE_YR']


In [7]:
drop_list=list(to_drop)
drop_list

['POP902',
 'POP903',
 'ETH13',
 'AGE902',
 'AGE903',
 'AGE904',
 'AGE905',
 'AGE906',
 'AGEC6',
 'CHILC1',
 'CHILC3',
 'HHAGE1',
 'HHAGE2',
 'HHAGE3',
 'HHN3',
 'HHN4',
 'HHN5',
 'HHN6',
 'MARR3',
 'HHP1',
 'HHP2',
 'DW2',
 'DW5',
 'DW6',
 'HV2',
 'HV4',
 'HU2',
 'HHD1',
 'HHD2',
 'HHD3',
 'HHD4',
 'HHD5',
 'HHD6',
 'HHD9',
 'HHD11',
 'ETHC2',
 'ETHC3',
 'ETHC4',
 'ETHC5',
 'ETHC6',
 'HVP1',
 'HVP2',
 'HVP3',
 'HVP4',
 'HVP5',
 'HVP6',
 'RHP1',
 'RHP2',
 'RHP3',
 'HUPA2',
 'HUPA5',
 'HUPA6',
 'RP1',
 'RP2',
 'RP3',
 'RP4',
 'IC2',
 'IC3',
 'IC4',
 'IC5',
 'IC11',
 'IC14',
 'IC15',
 'IC16',
 'IC17',
 'IC18',
 'IC19',
 'IC20',
 'IC21',
 'IC22',
 'IC23',
 'HHAS1',
 'HHAS4',
 'TPE4',
 'LFC2',
 'LFC3',
 'LFC4',
 'LFC5',
 'EIC1',
 'OEDC1',
 'OEDC2',
 'OEDC3',
 'SEC4',
 'AFC2',
 'AFC3',
 'AFC5',
 'LSC2',
 'LSC3',
 'VOC2',
 'VOC3',
 'HC5',
 'HC6',
 'HC7',
 'HC8',
 'HC21',
 'NUMPROM',
 'CARDGIFT',
 'FIRSTDATE_YR']

In [8]:
data=data.drop(drop_list, axis=1)
data

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_MM
0,0,60.000000,5,9,0,0,39,34,18,10,...,1,37,12,92,8,94,2,95,12,11
1,1,46.000000,6,9,16,0,15,55,11,6,...,1,52,2,93,10,95,12,95,12,10
2,1,61.611649,3,1,2,0,20,29,33,6,...,1,0,2,91,11,92,7,95,12,1
3,0,70.000000,1,4,2,0,23,14,31,3,...,1,28,1,87,11,94,11,95,12,2
4,0,78.000000,3,2,60,1,28,9,53,26,...,1,20,1,93,10,96,1,96,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,1,61.611649,5,9,0,14,36,47,11,7,...,1,0,2,96,2,96,2,96,2,2
95408,1,48.000000,7,9,1,0,31,43,19,4,...,1,50,1,96,3,96,3,96,3,3
95409,1,60.000000,5,9,0,0,18,46,20,7,...,1,38,1,96,3,95,1,96,10,10
95410,0,58.000000,7,9,0,0,28,35,20,9,...,1,40,5,90,11,96,8,97,1,12


In [9]:
print(data.isnull().sum())
print(data.isnull().sum())

TCODE           0
AGE             0
INCOME          0
WEALTH1         0
HIT             0
               ..
MAXRDATE_YR     0
MAXRDATE_MM     0
LASTDATE_YR     0
LASTDATE_MM     0
FIRSTDATE_MM    0
Length: 239, dtype: int64
TCODE           0
AGE             0
INCOME          0
WEALTH1         0
HIT             0
               ..
MAXRDATE_YR     0
MAXRDATE_MM     0
LASTDATE_YR     0
LASTDATE_MM     0
FIRSTDATE_MM    0
Length: 239, dtype: int64


In [10]:
#Procesamos data numerica y categorica
num = data.select_dtypes("number")
cat = data.select_dtypes("object")

In [11]:
#label enconcer para la darta categorica
cat=cat.apply(preprocessing.LabelEncoder().fit_transform)
cat

,STATE,HOMEOWNR,GENDER,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A
0,3,0,0,0,1,2,3
1,0,0,1,0,3,0,2
2,7,1,1,0,1,2,1
3,0,1,0,0,1,2,1
4,1,0,0,0,2,0,2
...,...,...,...,...,...,...,...
95407,11,0,1,0,3,2,0
95408,8,0,1,0,2,0,0
95409,5,0,1,0,1,1,0
95410,0,0,0,0,2,0,0


In [12]:
#Vamos a trabajar con el Target b
X=np.concatenate([num,cat], axis=1)
X=pd.DataFrame(X)
y1=target['TARGET_B']

In [13]:
#Aplicamos SMOTE para datavy target 1
upsampling1 = SMOTE()
X_smote,y_smote = upsampling1.fit_resample(X,y1)
y_smote.value_counts()

0    90569
1    90569
Name: TARGET_B, dtype: int64

In [14]:
#Aplicamos Random Forest
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.25, random_state=0)
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [ ]:
param_grid = {
       'n_estimators': [80, 150],
       'min_samples_split': [2, 4],
       'min_samples_leaf' : [1, 2],
       'max_features': ['sqrt']
           }
clf = RandomForestClassifier(random_state=100)
grid_search = GridSearchCV(clf, param_grid, cv=5,return_train_score=True,n_jobs=-1)
grid_search.fit(X_train,y_train)
grid_search.best_params_ #To check the best set of parameters returned"

In [ ]:
clf = RandomForestClassifier(random_state=100, max_features='sqrt', min_samples_leaf=1, min_samples_split=4, n_estimators=150)
cross_val_scores = cross_val_score(clf, X_train, y_train, cv=10)
print(np.mean(cross_val_scores))

In [ ]:

# codigo para guardar en mi archivo\n",
# Following code snippet will generate sorted dataframe with feature name and it's p-value. \n",
    
# funcion para obtener una lista del p-value
    
    # d = {}
    # for i in X.columns.tolist():
    # d[f'{i}'] = model_ols.pvalues[i]
    
    # df_pvalue= pd.DataFrame(d.items(), columns=['Var_name', 'p-Value']).sort_values(by = 'p-Value').reset_index(drop=True)


In [ ]:
#Vamos a poner en una lista alternativa qué columnas no cumplen
#con un máximo de nulos propuestos. Supongamos que nuestro umbral/threshold es de 7
#columns_above_threshold=df_pvalue[df_pvalue['p-Value']>0.7]
#print(columns_above_threshold)